<a href="https://colab.research.google.com/github/PiotrMaciejKowalski/kurs-analiza-danych-2022/blob/main/Tydzie%C5%84%206/Wprowadzenie_do_Apache_Spark.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Ustawiamy 

In [1]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null

In [6]:
!wget -q ftp.ps.pl/pub/apache/spark/spark-3.1.2/spark-3.1.2-bin-hadoop2.7.tgz

In [8]:
!tar xf spark-3.1.2-bin-hadoop2.7.tgz

In [9]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.1.2-bin-hadoop2.7"

In [10]:
!pip install -q findspark

import findspark
findspark.init()

In [12]:
!pip install pyspark

     |████████████████████████████████| 281.4 MB 30 kB/s 
     |████████████████████████████████| 198 kB 48.2 MB/s 
  Created wheel for pyspark: filename=pyspark-3.2.1-py2.py3-none-any.whl size=281853642 sha256=d45b1279e5bc888d358411efd0e4e7bfe58a3bdc4f18e25138bb6188255f16b9
  Stored in directory: /root/.cache/pip/wheels/9f/f5/07/7cd8017084dce4e93e84e92efd1e1d5334db05f2e83bcef74f
Successfully built pyspark


# Utworzenie sesji z pyspark

In [11]:
from pyspark.sql import SparkSession
spark = SparkSession.builder\
        .master("local")\
        .appName("Colab")\
        .config('spark.ui.port', '4050')\
        .getOrCreate()

# Apache Spark 

**Apache Spark** to zunifikowany silnik do obliczeń rozproszonych na licencji open-source. Oferuje interfejs pozwalający na programowanie obliczeń na klastrach z domyślną paralelizacją oraz odpornością na awarie.

Ze Sparkiem pracować można w Scali, Pythonie, Javie oraz R.

Jego przewaga nad model Map-Reduce Hadoopa polega na unikaniu zapisów na hdfs tak długo jak to możliwe - i posługiwaniu się RAMem nodów jak długo go wystarcza.

**Komponenty Sparka:**

<img src="spark-stack.png" alt="title" width="500"/>

* Spark "core" - podstawa Sparka z podstawową abstrakcją danych nazywaną RDD
* Spark SQL - komponent pozwalający na operowanie na ustrukturyzowanych danych z wykorzystaniem operacji znanych z SQL - łatwy w użyciu
* Spark MLlib - komponent zawierający algorytmy ML dostępne w Sparku - ML na skalę klastrów
* Spark Streaming - moduł pozwalający na pracę ze strumnieniami danych
* Spark GraphX - komponent do pracy z grafami

**Architektura Sparka:**

<img src="cluster-overview.png" alt="title" width="500"/>

* driver - proces uruchamiający główną funkcję aplikacji i tworzący SparkContext
* executor(y) - proces uruchomiony dla aplikacji w węźle roboczym (worker node), który uruchamia zadania i przechowuje dane w pamięci lub na dysku. Każda aplikacja ma własne executory
* cluster manager - dostępne opcje: YARN, Mesos, Kubernetes, Standalone

**SparkSession:**
* wprowadzony w Spark 2.0
* składa się ze SparkContextu, SQLContextu oraz HiveContext
* zwykle nazywany w kodzie `spark`
* kroki niezbędne do utworzenia SparkSession w pySparku:

> from pyspark.sql import SparkSession  
> spark = SparkSession.builder.getOrCreate()

## RDD

Podstawowym formatem danych (coś jak tabela w db) jest RDD. Skrót rozwija się następująco:
* R - resilient (elastyczny)
* D - distributed
* D - dataset

Model RDD jest napisany w sposób wspierający przekstrzałcenia Map-Reduce jako domyślny. W związku z powyższym wykazuje się następującymi własnościami:


* immutable - każdy obiekt jest niezmienniczy. Chcesz coś zmienić - musisz utworzyć nowy rdd
* in-memory - przetwarzany głównie w RAM
* lazy evaluated - silnik obliczeniowy wykonuje obliczenia dopiero gdy okażą się konieczne.
* parallel - współbieżny 

Z RDD stowarzyszone są dwa rodzaje czynności:
* akcje, oraz
* transformacje

### Transformacje 

Modelują czynności jakie możemy chcieć wykonywać na danych. Przekształcenia (map), redukcje (reduce), filtry (filter). Mają charakter opisu skąd się biorą pewne wartości. W naszym ujęciu mogą odpowiadać funkcjom mapper, reducer i podobnym. 

_Dla osób, które kojarzą paradygmat funkcyjny programowania - można dodać, że transformacje dotyczą funkcji czystych._

### Akcje

Modelują czynności z uwagi na wynik jaki oczekujemy. Wyświetl, zapisz, wyszukaj. Mają charakter silnie połączony z wynikiem działania. W naszym ujęciu pasują częściowo pod steps, a jeszcze bardziej pod `mrjob.run()`.

_Dla osób, które kojarzą paradygmat funkcyjny programowania - można dodać, że akcje dotyczą funkcji z efektem ubocznym._